In [1]:
# %%
import pandas as pd
from pathlib import Path

# -----------------------------------------
# DOSYA YOLLARI – ihtiyaca göre güncelle
energy_xlsx  = Path(r"C:\Users\alpgu\OneDrive\Desktop\DSA PROJECFT\ikitelli_daily_energy.xlsx")       # Üretim (Gün | Ay | Yıl | Üretim)
weather_xlsx = Path(r"C:\Users\alpgu\OneDrive\Desktop\DSA PROJECFT\daily_weather_2018-05_2019-05.xlsx")      # Hava   (Gün | Ay | Yıl | CloudCover_% | Temp_mean_C)
out_xlsx     = Path("train_data.xlsx")         # Birleşik çıktı
# -----------------------------------------

def add_date_col(df: pd.DataFrame) -> pd.DataFrame:
    """Gün-Ay-Yıl’dan güvenli şekilde Date sütunu üretir."""

    # 0) Ay adlarını numaraya map et (gerekiyorsa)
    if df["Ay"].dtype == object:
        month_map = {
                     "January":1,"February":2,"March":3,"April":4,"May":5,"June":6,
                     "July":7,"August":8,"September":9,"October":10,"November":11,"December":12}
        df["Ay"] = df["Ay"].str.strip().map(month_map)

    # Sayıya çevir, hatalıları NaN yap
    df["Ay"]  = pd.to_numeric(df["Ay"],  errors="coerce")
    df["Gün"] = pd.to_numeric(df["Gün"], errors="coerce")
    df["Yıl"] = pd.to_numeric(df["Yıl"], errors="coerce")

    # Eksikleri at
    df = df.dropna(subset=["Gün", "Ay", "Yıl"])


    with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,   # istersek tüm sütunları da göster
                       'display.expand_frame_repr', False):  # her satırı tek satırda tut
        print(df)

    print("Toplam satır:", len(df))
    # Int'e çevir
    df[["Gün","Ay","Yıl"]] = df[["Gün","Ay","Yıl"]].astype(int)

    # ---- Kritik SATIR (yeni) ----
    df["Date"] = pd.to_datetime(dict(year=df["Yıl"],
                                     month=df["Ay"],
                                     day=df["Gün"]))
    # ------------------------------
    return df
# 1) Dosyaları oku
df_energy  = pd.read_excel(energy_xlsx)
df_weather = pd.read_excel(weather_xlsx)

# 2) Tarih sütununu ekle
df_energy  = add_date_col(df_energy)
df_weather = add_date_col(df_weather)

# 3) INNER JOIN → yalnızca ortak günler
merged = (df_energy[["Date", "Üretim (kWh)"]]
          .merge(df_weather[["Date", "CloudCover_%", "Temp_mean_C"]],
                 on="Date", how="inner")
          .sort_values("Date")
          .reset_index(drop=True))

print("Son satır sayısı:", len(merged))

# Date sütununu 'YYYY-MM-DD' stringine çevir
merged["Date"] = merged["Date"].dt.strftime("%Y-%m-%d")

# (Opsiyonel) Date'i ilk sütuna al
cols = ["Date"] + [c for c in merged.columns if c != "Date"]
merged = merged[cols]

# 4) Excel çıktısı
merged.to_excel(out_xlsx, index=False)
print("✓ Birleşik veri kaydedildi →", out_xlsx)

     Gün  Ay   Yıl  Üretim (kWh)
0      1   5  2018       6400.00
1      2   5  2018       5670.00
2      3   5  2018       6770.00
3      4   5  2018       5210.00
4      5   5  2018       1920.00
5      6   5  2018       1700.00
6      7   5  2018       1500.00
7      8   5  2018       4049.00
8      9   5  2018       2920.00
9     10   5  2018       4270.00
10    11   5  2018       5070.00
11    12   5  2018       6960.00
12    13   5  2018       5668.00
13    14   5  2018       7190.00
14    15   5  2018       6360.00
15    16   5  2018       5840.00
16    17   5  2018       5370.00
17    18   5  2018       5830.00
18    19   5  2018       5890.00
19    20   5  2018       5140.00
20    21   5  2018       6606.00
21    22   5  2018       1783.00
22    23   5  2018       5580.00
23    24   5  2018       6522.00
24    25   5  2018       3231.00
25    26   5  2018       6352.00
26    27   5  2018       2182.00
27    28   5  2018       4322.00
28    29   5  2018       3009.99
29    30  